# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Aug 19 2022 10:27AM,244549,10,0085941656,ISDIN Corporation,Released
1,Aug 19 2022 10:27AM,244549,10,0085941659,ISDIN Corporation,Released
2,Aug 19 2022 10:27AM,244549,10,0085942500,ISDIN Corporation,Released
3,Aug 19 2022 10:27AM,244549,10,0085942513,ISDIN Corporation,Released
4,Aug 19 2022 10:27AM,244548,19,8414233,"Alexza Pharmaceuticals, Inc. DBA RHL",Released
5,Aug 19 2022 10:18AM,244546,21,624724,"NBTY Global, Inc.",Released
6,Aug 19 2022 10:16AM,244545,21,624706,"NBTY Global, Inc.",Released
7,Aug 19 2022 9:43AM,244544,16,8419177,Sartorius Bioprocess Solutions,Released
8,Aug 19 2022 9:43AM,244544,16,8419181,Sartorius Bioprocess Solutions,Released
9,Aug 19 2022 9:43AM,244544,16,8419180,Sartorius Bioprocess Solutions,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
13,244544,Released,5
14,244545,Released,1
15,244546,Released,1
16,244548,Released,1
17,244549,Released,4


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
244544,NaN,5.0
244545,NaN,1.0
244546,NaN,1.0
244548,NaN,1.0
244549,NaN,4.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
244458,1.0,0.0
244491,0.0,2.0
244509,0.0,1.0
244517,0.0,2.0
244519,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
244458,1,0
244491,0,2
244509,0,1
244517,0,2
244519,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,244458,1,0
1,244491,0,2
2,244509,0,1
3,244517,0,2
4,244519,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,244458,1,
1,244491,,2
2,244509,,1
3,244517,,2
4,244519,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Aug 19 2022 10:27AM,244549,10,ISDIN Corporation
4,Aug 19 2022 10:27AM,244548,19,"Alexza Pharmaceuticals, Inc. DBA RHL"
5,Aug 19 2022 10:18AM,244546,21,"NBTY Global, Inc."
6,Aug 19 2022 10:16AM,244545,21,"NBTY Global, Inc."
7,Aug 19 2022 9:43AM,244544,16,Sartorius Bioprocess Solutions
12,Aug 19 2022 9:39AM,244543,10,Bio-PRF
18,Aug 19 2022 9:10AM,244540,15,"Mizner Bioscience, LLC"
21,Aug 19 2022 9:03AM,244538,15,"Alliance Pharma, Inc."
37,Aug 19 2022 8:15AM,244531,10,ISDIN Corporation
52,Aug 19 2022 8:12AM,244530,10,ISDIN Corporation


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Aug 19 2022 10:27AM,244549,10,ISDIN Corporation,,4
1,Aug 19 2022 10:27AM,244548,19,"Alexza Pharmaceuticals, Inc. DBA RHL",,1
2,Aug 19 2022 10:18AM,244546,21,"NBTY Global, Inc.",,1
3,Aug 19 2022 10:16AM,244545,21,"NBTY Global, Inc.",,1
4,Aug 19 2022 9:43AM,244544,16,Sartorius Bioprocess Solutions,,5
5,Aug 19 2022 9:39AM,244543,10,Bio-PRF,,6
6,Aug 19 2022 9:10AM,244540,15,"Mizner Bioscience, LLC",,3
7,Aug 19 2022 9:03AM,244538,15,"Alliance Pharma, Inc.",11,5
8,Aug 19 2022 8:15AM,244531,10,ISDIN Corporation,15,
9,Aug 19 2022 8:12AM,244530,10,ISDIN Corporation,,7


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Aug 19 2022 10:27AM,244549,10,ISDIN Corporation,4,
1,Aug 19 2022 10:27AM,244548,19,"Alexza Pharmaceuticals, Inc. DBA RHL",1,
2,Aug 19 2022 10:18AM,244546,21,"NBTY Global, Inc.",1,
3,Aug 19 2022 10:16AM,244545,21,"NBTY Global, Inc.",1,
4,Aug 19 2022 9:43AM,244544,16,Sartorius Bioprocess Solutions,5,
5,Aug 19 2022 9:39AM,244543,10,Bio-PRF,6,
6,Aug 19 2022 9:10AM,244540,15,"Mizner Bioscience, LLC",3,
7,Aug 19 2022 9:03AM,244538,15,"Alliance Pharma, Inc.",5,11
8,Aug 19 2022 8:15AM,244531,10,ISDIN Corporation,,15
9,Aug 19 2022 8:12AM,244530,10,ISDIN Corporation,7,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Aug 19 2022 10:27AM,244549,10,ISDIN Corporation,4,
1,Aug 19 2022 10:27AM,244548,19,"Alexza Pharmaceuticals, Inc. DBA RHL",1,
2,Aug 19 2022 10:18AM,244546,21,"NBTY Global, Inc.",1,
3,Aug 19 2022 10:16AM,244545,21,"NBTY Global, Inc.",1,
4,Aug 19 2022 9:43AM,244544,16,Sartorius Bioprocess Solutions,5,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Aug 19 2022 10:27AM,244549,10,ISDIN Corporation,4.0,NaN
1,Aug 19 2022 10:27AM,244548,19,"Alexza Pharmaceuticals, Inc. DBA RHL",1.0,NaN
2,Aug 19 2022 10:18AM,244546,21,"NBTY Global, Inc.",1.0,NaN
3,Aug 19 2022 10:16AM,244545,21,"NBTY Global, Inc.",1.0,NaN
4,Aug 19 2022 9:43AM,244544,16,Sartorius Bioprocess Solutions,5.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Aug 19 2022 10:27AM,244549,10,ISDIN Corporation,4.0,0.0
1,Aug 19 2022 10:27AM,244548,19,"Alexza Pharmaceuticals, Inc. DBA RHL",1.0,0.0
2,Aug 19 2022 10:18AM,244546,21,"NBTY Global, Inc.",1.0,0.0
3,Aug 19 2022 10:16AM,244545,21,"NBTY Global, Inc.",1.0,0.0
4,Aug 19 2022 9:43AM,244544,16,Sartorius Bioprocess Solutions,5.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,1467139,19.0,27.0
15,489078,8.0,11.0
16,978071,10.0,0.0
19,489057,2.0,0.0
21,489091,2.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,1467139,19.0,27.0
1,15,489078,8.0,11.0
2,16,978071,10.0,0.0
3,19,489057,2.0,0.0
4,21,489091,2.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,19.0,27.0
1,15,8.0,11.0
2,16,10.0,0.0
3,19,2.0,0.0
4,21,2.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,19.0
1,15,Released,8.0
2,16,Released,10.0
3,19,Released,2.0
4,21,Released,2.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,16,19,21
Status,,,,,
Executing,27.0,11.0,0.0,0.0,0.0
Released,19.0,8.0,10.0,2.0,2.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,16,19,21
0,Executing,27.0,11.0,0.0,0.0,0.0
1,Released,19.0,8.0,10.0,2.0,2.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,16,19,21
0,Executing,27.0,11.0,0.0,0.0,0.0
1,Released,19.0,8.0,10.0,2.0,2.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()